In [2]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from scipy.fftpack import fft2, ifft2
import cupy as cp
from cupyx.scipy.fftpack import fft2 as cu_fft2
from cupyx.scipy.fftpack import ifft2 as cu_ifft2

In [3]:
def P(x, y, z, k):
    """ Fresnel propagator """
    norm = 1
    osn = np.exp(-(x**2 + y**2)/(2*z))
    return norm*osn

In [4]:
def P_gpu(x, y, z, k):
    """ Fresnel propagator """
    norm = 1
    osn = cp.exp(-(x**2 + y**2)/(2*z))
    return norm*osn

In [5]:
def ft2d_cpu(array, dx, dy):
    """
    Calculate normalized Fourier transform of the input 'array';
    return 'ft_array' of the same size
    """
    n, m = np.shape(array)
    k, l = np.arange(0, n), np.arange(0, m)
    c_k = np.exp(1j * np.pi * k * (1-1/n))
    c_l = np.exp(1j * np.pi * l * (1-1/m))
    C_l, C_k = np.meshgrid(c_l, c_k)
    C_kl = np.multiply(C_l, C_k)
    A_n = np.exp(1j * np.pi * (1 - 1/(2*n) - n/2))
    A_m = np.exp(1j * np.pi * (1 - 1/(2*m) - m/2))
    ft_array = dx * dy * A_n * A_m * np.multiply(C_kl, fft2(np.multiply(array, C_kl)))

    return ft_array

In [6]:
def ift2d_cpu(array, dx, dy):
    """
    Calculate normalized Fourier transform of the input 'array';
    return 'ft_array' of the same size
    """
    n, m = np.shape(array)
    k, l = np.arange(0, n), np.arange(0, m)
    c_k = np.exp(-1j * np.pi * k * (1-1/n))
    c_l = np.exp(-1j * np.pi * l * (1-1/m))
    C_l, C_k = np.meshgrid(c_l, c_k)
    C_kl = np.multiply(C_l, C_k)
    A_n = np.exp(-1j * np.pi * (1 - 1/(2*n) - n/2))
    A_m = np.exp(-1j * np.pi * (1 - 1/(2*m) - m/2))
    ft_array = 1/dx * 1/dy * A_n * A_m * np.multiply(C_kl, ifft2(np.multiply(array, C_kl)))

    return ft_array

In [7]:
def ft2d_gpu(array, dx, dy):
    
    n, m = cp.shape(array)
    k, l = cp.arange(0, n), cp.arange(0, m)
    c_k = cp.exp(1j * cp.pi * k * (1-1/n))
    c_l = cp.exp(1j * cp.pi * l * (1-1/m))
    C_l, C_k = cp.meshgrid(c_l, c_k)
    C_kl = cp.multiply(C_l, C_k)
    A_n = cp.exp(1j * np.pi * (1 - 1/(2*n) - n/2))
    A_m = cp.exp(1j * np.pi * (1 - 1/(2*m) - m/2))
    ft_array = dx * dy * A_n * A_m * np.multiply(C_kl, cu_fft2(cp.multiply(array, C_kl)))

    return ft_array

In [ ]:
def ift2d_gpu(array, dx, dy):
    
    n, m = cp.shape(array)
    k, l = cp.arange(0, n), cp.arange(0, m)
    c_k = cp.exp(-1j * cp.pi * k * (1-1/n))
    c_l = cp.exp(-1j * cp.pi * l * (1-1/m))
    C_l, C_k = cp.meshgrid(c_l, c_k)
    C_kl = cp.multiply(C_l, C_k)
    A_n = cp.exp(-1j * np.pi * (1 - 1/(2*n) - n/2))
    A_m = cp.exp(-1j * np.pi * (1 - 1/(2*m) - m/2))
    ft_array = 1/dx * 1/dy * A_n * A_m * np.multiply(C_kl, cu_ifft2(cp.multiply(array, C_kl)))
    
    return ft_array

In [8]:
N = 2**10
dx = 0.1
dq = 2*np.pi/(dx*N) # step of reciprocal-space
lb, rb = -(N-1)/2, N/2
barrx = np.arange(lb, rb) # base array
x, qx = barrx*dx, barrx*dq # real and reciprocal spaces

M = 2**10
dy = 0.1
dq = 2*np.pi/(dx*M) # step of reciprocal-space
lb, rb = -(M-1)/2, M/2
barry = np.arange(lb, rb) # base array
y, qy = barry*dy, barry*dq # real and reciprocal spaces

# y = stats.norm.pdf(x, 0, 1)
# plt.plot(q, ft1d_cpu(y, dx).imag)
X, Y = np.meshgrid(x, y)
X_gpu, Y_gpu = cp.array(X), cp.array(Y)
QX, QY = np.meshgrid(qx, qy)

In [9]:
gauss_pdf = P(X, Y, z=1, k=1)
gauss_pdf_gpu = P_gpu(X_gpu, Y_gpu, z=1, k=1)
print(type(gauss_pdf[0][0]))

<class 'numpy.float64'>


In [10]:
cp.arange(1, 10)

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

In [ ]:
# sigma = 1
# rv = stats.multivariate_normal([0, 0], [[sigma**2, 0], [0, sigma**2]])
# pos = np.dstack((X, Y))
# # pos = [0, 0]
# gauss_pdf = rv.pdf(pos)

# gauss_pdf_gpu = cp.array(gauss_pdf)
# print(type(gauss_pdf_gpu.get()))

# fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
# ax.plot_surface(X, Y, rv.pdf(pos))

fig1, ax1 = plt.subplots(1,1)
pcol = plt.pcolor(X, Y, gauss_pdf, cmap="bone")
ax1.set_xlim(-5, 5)
ax1.set_ylim(-5, 5)
plt.colorbar(pcol)

In [ ]:
fig2, ax2 = plt.subplots(1,1)
gauss_fft_gpu = ft2d_gpu(gauss_pdf_gpu, dx, dy)
pcol2 = plt.pcolormesh(QX, QY, gauss_fft_gpu.get().real, cmap="bone")
ax2.set_xlim(-5, 5)
ax2.set_ylim(-5, 5)
plt.colorbar(pcol2)

In [ ]:
fig2, ax2 = plt.subplots(1,1)
gauss_fft_gpu = ft2d_gpu(gauss_pdf_gpu, dx, dy)
gauss_fft = ft2d_cpu(gauss_pdf, dx, dy)
print(type(gauss_fft_gpu.get()[0][0]))
print(type(gauss_fft[0][0]))
# pcol2 = plt.pcolormesh(X, Y, gauss_pdf_gpu.get() - gauss_pdf, cmap="bone")
pcol2 = plt.pcolormesh(X, Y, gauss_fft.imag, cmap="bone")
# ax2.set_xlim(-5, 5)
# ax2.set_ylim(-5, 5)
plt.colorbar(pcol2)

In [ ]:
pcol2 = plt.pcolormesh(X, Y, gauss_fft_gpu.get().imag, cmap="bone")
# ax2.set_xlim(-5, 5)
# ax2.set_ylim(-5, 5)
plt.colorbar(pcol2)

In [ ]:
%%timeit
gauss_fft_gpu = ft2d_gpu(gauss_pdf_gpu, dx, dy)

In [ ]:
%%timeit
gauss_fft = ft2d_cpu(gauss_pdf, dx, dy)

In [ ]:
fig2, ax2 = plt.subplots(1,1)
gauss_fft = ft2d_cpu(gauss_pdf, dx, dy)
pcol2 = plt.pcolormesh(X, Y, ift2d_cpu(gauss_fft, dx, dy).imag, cmap="bone")
# ax2.set_xlim(-5, 5)
# ax2.set_ylim(-5, 5)
plt.colorbar(pcol2)

In [ ]:
fig2, ax2 = plt.subplots(1,1)
pcol2 = plt.pcolormesh(X, Y, gauss_fft_gpu.get().imag, cmap="bone")
ax2.set_xlim(-5, 5)
ax2.set_ylim(-5, 5)
plt.colorbar(pcol2)